In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Caricare il path del file delle detection con la specifica soglia di confidence (0.6 o 0.7), quindi importarlo

In [2]:
# upload
path_data = '/content/MOT17-05-DPM-DETR06.txt' # path di esempio, sto caricando nel content
data_preloaded = []
with open(path_data, 'r') as f:
    for row in f:
        frame, bboxes, confidences = row.split(',')
        frame = int(frame)
        x1, x2, y1, y2 = bboxes.strip('[] ').split()
        x1 = float(x1)
        x2 = float(x2)
        y1 = float(y1)
        y2 = float(y2)
        conf = float(confidences.strip())
        detection_frame = [frame,[x1,x2,y1,y2], conf]
        data_preloaded.append(detection_frame)

In [ ]:
# data_preloaded

Adesso devo adattarli alle mie detection, quindi mi creo un dizionario di detection dove la chiave corrisponde al frame e il valore invece la lista di detection+confidences per quel determinato frame

In [3]:
final_detection = {}
for data in data_preloaded:
    if data[0] not in final_detection.keys():
        final_detection[data[0]] = []
    final_detection[data[0]].append([data[1], data[2]])

In [4]:
len(final_detection)

837

Adesso quindi se volessi estrarre le detection del primo frame:

In [7]:
detections_frame_1 = final_detection[1]
detections_frame_1

[[[308.0633, 214.77393, 318.02454, 242.05516], 0.6666003465652466],
 [[308.2305, 213.24954, 318.4695, 241.51703], 0.7635806798934937],
 [[271.06854, 202.00267, 285.7063, 243.01091], 0.8880743980407715],
 [[266.40875, 202.52731, 282.27567, 244.7044], 0.9473226070404053],
 [[119.37081, 155.9093, 199.8064, 328.23328], 0.9997299313545227],
 [[19.042439, 146.2177, 93.47551, 338.71338], 0.9987555742263794],
 [[57.858868, 154.56133, 92.59082, 316.49045], 0.8556430339813232],
 [[250.73053, 201.42151, 263.4828, 247.09451], 0.9794502258300781]]

In [12]:
for detection, confidence in final_detection[1]:
    print(f'Detection: {detection}; Confidence: {confidence}')

Detection: [308.0633, 214.77393, 318.02454, 242.05516]; Confidence: 0.6666003465652466
Detection: [308.2305, 213.24954, 318.4695, 241.51703]; Confidence: 0.7635806798934937
Detection: [271.06854, 202.00267, 285.7063, 243.01091]; Confidence: 0.8880743980407715
Detection: [266.40875, 202.52731, 282.27567, 244.7044]; Confidence: 0.9473226070404053
Detection: [119.37081, 155.9093, 199.8064, 328.23328]; Confidence: 0.9997299313545227
Detection: [19.042439, 146.2177, 93.47551, 338.71338]; Confidence: 0.9987555742263794
Detection: [57.858868, 154.56133, 92.59082, 316.49045]; Confidence: 0.8556430339813232
Detection: [250.73053, 201.42151, 263.4828, 247.09451]; Confidence: 0.9794502258300781


Poi quindi nella mia funzione:

In [13]:
def process_image_folder_preloaded(detections_loaded, folder_path, frame_size=(640, 360), detection_interval=1, frame_limit_flag = False, limit=5):
    tracker = Tracker()
    frame_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))])
    frame_count = 0

    detections_frame = []

    print('Fine preload')
    for frame_file in frame_files:
        if frame_limit_flag and frame_count > limit:
            break
        frame = Image.open(frame_file)

        if frame_count % detection_interval == 0:
            # confidences, detections = detect_pedestrians(im=frame, model=model)
            detections_frame = detections_loaded[frame_count]
        detections = []
        confidences = []
        for detection, confidence in detections_frame:
            detections.append(detection)
            confidences.append(confidence)

        # quindi da qua in poi non cambia più nulla rispetto a prima
        tracker.update_tracker(confidences, detections, frame, vgg16, theshold_det_track = 0.4, theshold_reid=0.4)

        actual_detections = [] # solo per print

        for track in tracker.trackers:
            if track['lost'] == 0:
                x1, y1, x2, y2 = map(int, track['bbox'])
                x = x1
                y = y1
                w = x2-x1
                h = y2-y1
                conf = track['conf']
                # poi format_detection deve essere stampato in un file
                format_detectetion = [frame_count, track['id'], x,y,w,h, track['conf'],-1,-1,-1]
                print(format_detectetion)
                actual_detections.append(format_detectetion) # solo per printing
                detections_frame.append(format_detectetion)
        print(f'Frame: {frame_count}')
        # plot_image_w_detections(frame, actual_detections)
        frame_count += 1
    return detections_frame